# Store kitti sequence

Create a sequence

In [1]:
# when you change a setting you have to restart the kernel (settings are only loaded once)
import os
#os.environ['STEREO_CONFIDENCE'] = '1.6'
os.environ['SEQUENCE_CONFIDENCE'] = '1.6'
#os.environ['PATCH_SIZE'] = '17'
FRAMECOUNT = 1101

import sys
sys.path.append('..')
from src.kitti import *
%matplotlib inline
import numpy as np
import glob
import urbg2o
import cv2


IMAGE_SIZE = 400

def load(file):
    keyframeids, frameids, poses = load_keyframes(file)
    return keyframeids, frameids, poses.reshape(poses.shape[0], 4, 4)

def load_csv(file):
    poses = np.genfromtxt(file, delimiter= " ")
    poses = np.hstack([poses, np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) ])
    return poses.reshape(poses.shape[0], 4, 4)

def transform(poses):
    points = np.repeat(np.array([[0,0,0,1.0]]), len(poses), axis=0) 
    for i in range(1, len(poses)):
        points[:i] = np.dot(points[:i], poses[i].T)
    return points
        
def transform2(poses):
    points = poses[:,:,3]
    return points
    
def plot_trajectory(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1])

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100

    return np.dot(points, t.T)
    

def plot_trajectory2(points):
    # remove y coordinate (we only use x and z to plot a view from above)
    points = points[:, [0,2,3]]
    # compute min and max
    pmin = points.min(axis=0)
    pmax = points.max(axis=0)
    maxxy = max(pmax[0] - pmin[0], pmax[1] - pmin[1]) * 1.01

    t = np.eye(3, dtype=np.float64)
    t[0,0] = IMAGE_SIZE / maxxy
    t[1,1] = -IMAGE_SIZE / maxxy
    t[0,2] = -pmin[0] * IMAGE_SIZE / maxxy
    t[1,2] = IMAGE_SIZE / maxxy + pmin[1] / maxxy + 100
    return np.dot(points, t.T)
    
def plot_trajectory3(points):
    coords = []
    minx = min([p[0] for p in points])
    maxx = max([p[0] for p in points])
    miny = min([p[2] for p in points])
    maxy = max([p[2] for p in points])
    maxxy = max(maxx - minx, maxy - miny)* 1.01

    for p in points:
        #print(p)
        x = IMAGE_SIZE * (p[0] - minx) / (maxxy)
        z = IMAGE_SIZE - IMAGE_SIZE * (p[2] - miny) / (maxxy)
        coords.append([x, z])
    return coords

def new_image():
    img = np.zeros((IMAGE_SIZE, IMAGE_SIZE, 3), np.uint8)
    img[:] = 255 #or img.fill(255)
    return img

def draw(coords, img, color = (0,0,0)):
    # Draw a black line with thickness of 2 px
    for i in range(1,len(coords)):
        cv2.line(img, (int(coords[i - 1][0]), int(coords[i - 1][1])), (int(coords[i][0]), int(coords[i][1])), color, 2)
    return img

Read the first 10 frames from KITTI sequence 00.

In [2]:
%%time
SEQUENCE=7
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_0'%SEQUENCE
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_1'%SEQUENCE

frame = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    frame.append(left_frame)

CPU times: user 4.62 ms, sys: 1.32 ms, total: 5.94 ms
Wall time: 4.93 ms


Add the frames to a sequence. By adjusting the [:] you can use only a slice of the list of frames.

In [3]:
seq = Sequence()
for f in ProgressBar()(frame[:50]):
    seq.add_frame(f, run_ba=False)

  2% (1 of 50) |                          | Elapsed Time: 0:00:00 ETA:  0:00:19

connected mappoints  0


 16% (8 of 50) |####                      | Elapsed Time: 0:00:01 ETA:  0:00:09

invalid speed keyframe 0 frame 8 speed 0.05547547817966936
 [[  9.95999144e-01  -4.07609529e-05   8.93627597e-02  -1.81468063e-02]
 [ -2.93936672e-04   9.99992992e-01   3.73221845e-03   6.27038737e-03]
 [ -8.93622856e-02  -3.74355338e-03   9.95992152e-01  -7.30242552e-01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.00000000e+00]]


 18% (9 of 50) |####                      | Elapsed Time: 0:00:02 ETA:  0:00:12

...

 96% (48 of 50) |######################## | Elapsed Time: 0:00:25 ETA:  0:00:00

invalid speed keyframe 40 frame 48 speed 3.050245032839027
 [[ 0.98660894 -0.01472485  0.16243761 -0.97965387]
 [ 0.0125789   0.9998196   0.01423153 -0.77198753]
 [-0.16261786 -0.01199767  0.98661618 -0.52975421]
 [ 0.          0.          0.          1.        ]]


 98% (49 of 50) |######################## | Elapsed Time: 0:00:26 ETA:  0:00:00

connected mappoints  0


100% (50 of 50) |#########################| Elapsed Time: 0:00:26 Time: 0:00:26


In [4]:
%%time
SEQUENCE=7
LEFTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_0'%SEQUENCE
RIGHTDIR = '/data/urbinn/datasets/kitti/sequences/%02d/image_1'%SEQUENCE

kframe = []
for filename in sorted(glob.glob(LEFTDIR + '/*')):
    left_frame = Frame(filename, RIGHTDIR)
    kframe.append(left_frame)

CPU times: user 5.61 ms, sys: 57 µs, total: 5.67 ms
Wall time: 4.96 ms


In [5]:
kseq = Sequence()
for f in ProgressBar()(kframe[:50]):
    kseq.add_keyframe(f, run_ba=False)

  2% (1 of 50) |                          | Elapsed Time: 0:00:00 ETA:  0:00:19

connected mappoints  0


  4% (2 of 50) |#                         | Elapsed Time: 0:00:00 ETA:  0:00:22

connected mappoints  98


  6% (3 of 50) |#                         | Elapsed Time: 0:00:01 ETA:  0:00:24

...

connected mappoints  0


 98% (49 of 50) |######################## | Elapsed Time: 0:00:52 ETA:  0:00:00

connected mappoints  0


100% (50 of 50) |#########################| Elapsed Time: 0:00:53 Time: 0:00:53


connected mappoints  0


Example of how to save a set of covisible keyframes, fixed keyframes, mappoints and edges to use for local bundle adjustment

In [6]:
print('Keyframes count\n', (len(seq.keyframes)), '\n')

def keyframes_to_np(keyframes):
    return np.hstack([ [ [ kf.keyframeid] for kf in keyframes ], [ [kf.frameid] for kf in keyframes ], [ kf.get_world_pose().flatten() for kf in keyframes ] ])

Keyframes count
 10 



In [7]:
cv_keyframes = [ seq.keyframes[f] for f in range(len(seq.keyframes)) ]
#cv_keyframes = get_covisible_keyframes(seq.keyframes[-1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

#cv_keyframes_BA[-1,2:] = cv_keyframes_BA[-1,2:] * 2
#print(cv_keyframes_BA[3])
results =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

#[f.frameid for f in cv_keyframes]
cv_keyframes_np[:, [5,13]].reshape((-1,2))

array([[ 0.        ,  0.        ],
       [ 0.0797279 , -0.78571803],
       [ 0.01082788, -1.51295057],
       [-0.09637543, -2.77760074],
       [-0.70003742, -4.50551005],
       [-1.52747023, -5.6931009 ],
       [-2.23052934, -6.30754764],
       [-3.61637593, -6.96596951],
       [-5.18553317, -7.35796517],
       [-8.76441131, -7.40643895]])

In [8]:
cv_keyframes = [ kseq.keyframes[f] for f in range(0,50) ]
#cv_keyframes = get_covisible_keyframes(seq.keyframes[-1])
mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

#cv_keyframes_BA[-1,2:] = cv_keyframes_BA[-1,2:] * 2
#print(cv_keyframes_BA[3])
results =  urbg2o.localBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

#[f.frameid for f in cv_keyframes]
cv_keyframes_np[:, [5,13]].reshape((-1,2))

array([[  0.00000000e+00,   0.00000000e+00],
       [  8.67249481e-03,  -9.06069810e-02],
       [  1.88055713e-02,  -1.88290844e-01],
       [  2.72520020e-02,  -2.85559938e-01],
       [  3.50679277e-02,  -3.86974325e-01],
       [  5.07425081e-02,  -5.03478664e-01],
       [  6.05318846e-02,  -6.31422815e-01],
       [  7.07196697e-02,  -7.72433960e-01],
       [  8.39171069e-02,  -9.24678163e-01],
       [  9.52740370e-02,  -1.08758364e+00],
       [  1.03557737e-01,  -1.26498413e+00],
       [  1.09445038e-01,  -1.45536173e+00],
       [  1.20367550e-01,  -1.64117497e+00],
       [  1.18383601e-01,  -1.84376769e+00],
       [  1.04635567e-01,  -2.04083051e+00],
       [  8.79640890e-02,  -2.24775947e+00],
       [  6.46288123e-02,  -2.46325941e+00],
       [  3.13193066e-02,  -2.68592453e+00],
       [ -8.41345411e-03,  -2.89970444e+00],
       [ -5.68836013e-02,  -3.11620292e+00],
       [ -1.16545631e-01,  -3.32839919e+00],
       [ -1.98288193e-01,  -3.55337033e+00],
       [ -

In [9]:
len(links) / 50

101.66

In [10]:
#links2 = [ l for l in links if l[1] == 2.0]
#[ l for l in links if int(l[0]) >= len(mappoints)]
#mid2 = [ int(m[0]) for m in mappoints ]
#[ l for l in links2 if int(l[0]) not in mid2 ]
cv_keyframes_BA[:, [5,13]].reshape((-1,2))

array([[  0.00000000e+00,   0.00000000e+00],
       [  8.67249481e-03,  -9.06069810e-02],
       [  1.53862079e-02,  -1.81592799e-01],
       [  1.83422697e-02,  -2.75980837e-01],
       [  2.23633946e-02,  -3.76359011e-01],
       [  2.95911296e-02,  -4.85673234e-01],
       [  3.25090991e-02,  -6.06682367e-01],
       [  3.32235315e-02,  -7.38557728e-01],
       [  3.53668456e-02,  -8.80569912e-01],
       [  3.17784837e-02,  -1.03513856e+00],
       [  2.67665483e-02,  -1.19966387e+00],
       [  2.14078435e-02,  -1.37691542e+00],
       [  1.06048912e-02,  -1.55395453e+00],
       [ -1.04003950e-03,  -1.74970552e+00],
       [ -3.03592772e-02,  -1.94003042e+00],
       [ -5.91711628e-02,  -2.14535190e+00],
       [ -9.72844156e-02,  -2.35714177e+00],
       [ -1.45133744e-01,  -2.57078640e+00],
       [ -1.97678612e-01,  -2.78598130e+00],
       [ -2.63673121e-01,  -2.99846149e+00],
       [ -3.45813467e-01,  -3.20342180e+00],
       [ -5.07428436e-01,  -3.32339079e+00],
       [ -

In [11]:

#cv_keyframes = get_covisible_keyframes(seq.keyframes[-1])
cv_keyframes = [ seq.keyframes[f] for f in range(len(seq.keyframes)) ]

mappoints = get_mappoints(cv_keyframes)
f_keyframes = get_fixed_keyframes(mappoints, cv_keyframes)[:0]

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

outliers =  urbg2o.outliersForLocalBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

cv_keyframes_np[:,[5,13]]

array([[ 0.        ,  0.        ],
       [ 0.0797279 , -0.78571803],
       [ 0.01082788, -1.51295057],
       [-0.09637543, -2.77760074],
       [-0.70003742, -4.50551005],
       [-1.52747023, -5.6931009 ],
       [-2.23052934, -6.30754764],
       [-3.61637593, -6.96596951],
       [-5.18553317, -7.35796517],
       [-8.76441131, -7.40643895]])

In [12]:
countsp = [0] * 50
counts = [0] * 50
for o in outliers:
    if o[2] > 6:
        counts[int(o[0])] += 1
for o in links:
    countsp[int(o[1])] += 1
for i in range(len(counts)):
    counts[i] /= countsp[i] + 0.001
counts

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [13]:
from sortedcontainers import SortedSet

inliers = { (int(pointid), int(frameid)) for frameid, pointid, chi2 in outliers if chi2 < 5.8 }
keptpoints = SortedSet({ l[0] for l in inliers })
keptframes = SortedSet({ l[1] for l in inliers })
#m_keptpoints = { p:i for i, p in enumerate(keptpoints) }
#m_keptframes = { p:i for i, p in enumerate(keptframes) }
cv_keyframes1 = [ f for f in cv_keyframes if f.keyframeid in keptframes]
fixed_keyframes1 = [ f for f in f_keyframes if f.keyframeid in keptframes]
mappoints1 = [ f for f in mappoints if int(f[0]) in keptpoints]
links1 = [ (l[0], l[1], l[2], l[3]) for l in links if (int(l[0]), int(l[1])) in inliers ]

In [14]:
len(mappoints), len(links)

(76, 152)

In [15]:
len(inliers) / len(links)

1.0

In [16]:
cv_keyframes1_np = keyframes_to_np(cv_keyframes1)
f_keyframes1_np = keyframes_to_np(fixed_keyframes1)
links1_np = np.array(links1, dtype=np.float64, order='f')

f_keyframes1 = np.asfortranarray(f_keyframes1_np)
cv_keyframes1_BA = np.asfortranarray(cv_keyframes1_np)
mappoints1 = np.asfortranarray(mappoints1)
links1 = np.array(links1_np, order='f')

results =  urbg2o.localBundleAdjustment(cv_keyframes1_BA, f_keyframes1, mappoints1, links1)



In [17]:
cv_keyframes1_BA[:,2:]

array([[  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.97576361e-01,  -2.29348128e-03,   6.95423935e-02,
          7.97279018e-02,   1.98112817e-03,   9.99987640e-01,
          4.56018670e-03,  -6.18679030e-03,  -6.95519927e-02,
         -4.41136206e-03,   9.97568574e-01,  -7.85718030e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  9.87204603e-01,  -1.05456578e-02,   1.59109586e-01,
         -2.35583047e-01,   9.35983962e-03,   9.99922570e-01,
          8.20041413e-03,   4.27444431e-02,  -1.59183745e-01,
         -6.60624637e-03,   9.87226870e-01,  -7.48027638e-01,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.0000

In [18]:

cv_keyframes = { seq.keyframes[i] for i in range(0,7,2) }
mappoints = get_mappoints(cv_keyframes)
f_keyframes = { }

cv_keyframes_np = keyframes_to_np(cv_keyframes)
f_keyframes_np = keyframes_to_np(f_keyframes)
mappoints_np = mappoints_to_np(mappoints)
links_np = links_to_np(mappoints)


f_keyframes = np.asfortranarray(f_keyframes_np)
cv_keyframes_BA = np.asfortranarray(cv_keyframes_np)
mappoints = np.asfortranarray(mappoints_np)
links = np.array(links_np, order='f')

# np.set_printoptions(threshold=np.nan)

# print('mappoints count  \n', (len(mappoints[:]), '\n') )
# print('mappoints \n', (mappoints[:]), '\n')
# print('links count  \n', (len(links[:]), '\n') )
# print('links \n', (links[:]), '\n')

# print('Keyframes \n', (cv_keyframes), '\n')
# print('Keyframes numpy \n', (cv_keyframes_np[:, 2 :]), '\n')
# print('mappoints len \n', len(mappoints_np), '\n')
# print('Fixed Keyframes \n',f_keyframes[:], '\n')

# print('Keyframes before BA \n', (cv_keyframes_BA[: , 2:]), '\n')
# print(len(cv_keyframes_BA), '\n')

outliers =  urbg2o.outliersForLocalBundleAdjustment(cv_keyframes_BA, f_keyframes, mappoints, links)

# print('Keyframes after BA \n', (cv_keyframes_BA[:, 2 :]), '\n')
# print(len(cv_keyframes_BA), '\n')

In [19]:
from sortedcontainers import SortedSet

inliers = { (int(pointid), int(frameid)) for frameid, pointid, chi2 in outliers if chi2 < 10 }
keptpoints = SortedSet({ l[0] for l in inliers })
keptframes = SortedSet({ l[1] for l in inliers })
m_keptpoints = { p:i for i, p in enumerate(keptpoints) }
m_keptframes = { p:i for i, p in enumerate(keptframes) }
cv_keyframes1 = [ f for i, f in enumerate(cv_keyframes) if i in keptframes]
fixed_keyframes1 = [ f for i, f in enumerate(f_keyframes) if i + len(cv_keyframes) in keptframes]
mappoints1 = [ f for i, f in enumerate(mappoints) if i in keptpoints]
links1 = [ (m_keptpoints[l[0]], m_keptframes[l[1]], l[2], l[3]) for l in links if (int(l[0]), int(l[1])) in inliers ]

In [20]:
len(inliers) / len(links)

0.5

In [21]:
cv_keyframes1_np = keyframes_to_np(cv_keyframes1)
f_keyframes1_np = keyframes_to_np(fixed_keyframes1)
links1_np = np.array(links1, dtype=np.float64, order='f')

f_keyframes1 = np.asfortranarray(f_keyframes1_np)
cv_keyframes1_BA = np.asfortranarray(cv_keyframes1_np)
mappoints1 = np.asfortranarray(mappoints1)
links1 = np.array(links1_np, order='f')

results =  urbg2o.localBundleAdjustment(cv_keyframes1_BA, f_keyframes1, mappoints1, links1)



In [22]:
cv_keyframes1_BA[:, 2:]

array([[  4.61600542e-01,  -1.69087287e-03,   8.87086287e-01,
         -2.23052934e+00,  -9.74562357e-03,   9.99928169e-01,
          6.97715276e-03,  -3.12361549e-02,  -8.87034364e-01,
         -1.18658665e-02,   4.61550906e-01,  -6.30754764e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00],
       [  1.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   1.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   1.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          1.00000000e+00]])